In [1]:
# Importing the required libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [3]:
# Loading Data
df = pd.read_csv('dataset.csv')

In [4]:
# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

In [5]:
# Sorting
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [6]:
# Time of day weighting based on observation
def get_time_weight(hour):
    if 8 <= hour < 9:
        return 0.7
    elif 9 <= hour < 11:
        return 1.0
    elif 11 <= hour < 14:
        return 1.2
    elif 14 <= hour <= 16:
        return 1.0
    else:
        return 1.0

# Traffic mapping
traffic_map = {'low': 0, 'medium': 1, 'high': 2}

# Vehicle mapping
vehicle_map = {'car': 1.0, 'bike': 0.85, 'truck': 1.2}

# Applying the capped growth per 30 minutes
def apply_capped_growth(prices, cap=3.0):
    smoothed = [prices[0]]
    for i in range(1, len(prices)):
        prev = smoothed[-1]
        raw = prices[i]
        capped = np.clip(raw, prev - cap, prev + cap)
        smoothed.append(capped)
    return smoothed

In [7]:
# Adding Time Column
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hour'] = df['Timestamp'].dt.hour

# Constants
base_price = 10
lambda_scale = 1.0
min_price = 5
max_price = 20

# Empty column to store Model 2 price
df['Price_DemandModel'] = np.nan

In [8]:
for lot_id, lot_df in df.groupby('SystemCodeNumber'):
    lot_df = lot_df.sort_values('Timestamp').copy()
    momentum = np.zeros(len(lot_df))

    lot_df['Momentum'] = 0.0 #initialize

    # Compute momentum per day
    for date, day_df in lot_df.groupby(lot_df['Timestamp'].dt.date):
        occ = day_df['Occupancy'].values
        cap = day_df['Capacity'].iloc[0]
        delta = np.diff(occ, prepend=occ[0]) / cap
        m = pd.Series(delta).rolling(3, min_periods=1).sum().fillna(0)
        lot_df.loc[day_df.index, 'Momentum'] = m.values

    # Map categorical features with .fillna()
    traffic_val = lot_df['TrafficConditionNearby'].map(traffic_map).fillna(0)
    vehicle_val = lot_df['VehicleType'].map(vehicle_map).fillna(1.0)
    time_weight = lot_df['Hour'].apply(get_time_weight)

    # Check if we have enough data to compute demand
    if len(lot_df) == 0:
        continue

    # Demand function
    demand_raw = (
        2.0 * lot_df['Momentum'] +
        1.0 * lot_df['QueueLength'] +
        -1.0 * traffic_val +
        1.0 * lot_df['IsSpecialDay'] +
        1.0 * vehicle_val
    )

    demand_weighted = demand_raw * time_weight

    # Normalize demand
    min_demand = demand_weighted.min()
    max_demand = demand_weighted.max()

    if max_demand == min_demand:
        norm_demand = np.zeros_like(demand_weighted)
    else:
        norm_demand = (demand_weighted - min_demand) / (max_demand - min_demand)

    # Final price calculation (raw demand-based)
    price = base_price * (1 + lambda_scale * norm_demand)

    # Clamp to $5–$20 as per problem statement
    price_clamped = np.clip(price, min_price, max_price)

    # Smooth the price to prevent sharp jumps (±$3.00 per 30-min step)
    price_smoothed = apply_capped_growth(price_clamped.tolist(), cap=3.0)

    # Assign smoothed prices to final column
    df.loc[lot_df.index, 'Price_DemandModel'] = price_smoothed

In [9]:
# Downloading the RESULTANT csv file
df.to_csv("demand_pricing_model.csv", index=False)

from google.colab import files
files.download("demand_pricing_model.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# BOKEH PLOT
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import output_notebook
output_notebook()

# Get list of all parking lots
unique_lots = df['SystemCodeNumber'].unique()
figures = []

# Loop through all lots
for lot in unique_lots:
    lot_df = df[df['SystemCodeNumber'] == lot].sort_values('Timestamp')

    source = ColumnDataSource(data={
        'x': lot_df['Timestamp'],
        'y': lot_df['Price_DemandModel']
    })

    p = figure(title=lot, x_axis_type='datetime', width=350, height=250)
    p.line('x', 'y', source=source, line_width=2, color='navy')
    p.scatter('x', 'y', source=source, size=4, color='navy', marker='circle')
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    p.title.text_font_size = '10pt'
    figures.append(p)

# Arrange plots in a 3-column grid
grid = gridplot([figures[i:i+3] for i in range(0, len(figures), 3)])
show(grid)

In [11]:
# Loading the Model 1 Result for Bokeh Comparison
from google.colab import files
uploaded = files.upload()

Saving momentum_pricing_model.csv to momentum_pricing_model.csv


In [12]:
# Model 2 Results
df_model2_clean = df[['Timestamp', 'SystemCodeNumber', 'Price_DemandModel']].copy()

# Model 1 Results
df_model1 = pd.read_csv('momentum_pricing_model.csv')

# Convert timestamps to datetime
df_model1['Timestamp'] = pd.to_datetime(df_model1['Timestamp'])
df_model2_clean['Timestamp'] = pd.to_datetime(df_model2_clean['Timestamp'])

# Merge: bring Momentum price into Model 2 data
df_combined = df_model2_clean.merge(
    df_model1[['Timestamp', 'SystemCodeNumber', 'Price_Momentum']],
    on=['Timestamp', 'SystemCodeNumber'],
    how='left'
)

In [14]:
# BOKEH Plot of Comparison between both the models...
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

unique_lots = df_combined['SystemCodeNumber'].unique()
figures = []

for lot in unique_lots:
    lot_df = df_combined[df_combined['SystemCodeNumber'] == lot].sort_values('Timestamp')

    if lot_df['Price_Momentum'].isnull().all() or lot_df['Price_DemandModel'].isnull().all():
        continue  # skip if no data

    source = ColumnDataSource(data={
        'x': lot_df['Timestamp'],
        'model1': lot_df['Price_Momentum'],
        'model2': lot_df['Price_DemandModel']
    })

    p = figure(title=lot, x_axis_type='datetime', width=350, height=250)

    p.line('x', 'model1', source=source, line_width=2, color='green', legend_label='Momentum')
    p.line('x', 'model2', source=source, line_width=2, color='orange', line_dash='dashed', legend_label='Demand')

    p.scatter('x', 'model1', source=source, size=4, color='green', marker='circle')
    p.scatter('x', 'model2', source=source, size=4, color='orange', marker='circle')

    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    p.title.text_font_size = '10pt'
    p.legend.label_text_font_size = '8pt'
    p.legend.location = 'top_left'

    figures.append(p)

# Display all plots in a 3-column grid
grid = gridplot([figures[i:i+3] for i in range(0, len(figures), 3)])
show(grid)